In [177]:
import numpy as np
import math
from collections import Counter

NUM_COLUMNS = 7
COLUMN_HEIGHT = 6
FOUR = 4
MINMAX_DEPTH = 2

# Board can be initiatilized with `board = np.zeros((NUM_COLUMNS, COLUMN_HEIGHT), dtype=np.byte)`
# Notez Bien: Connect 4 "columns" are actually NumPy "rows"


Basic Functions

In [178]:
def valid_moves(board):
    """Returns columns where a disc may be played"""
    return [n for n in range(NUM_COLUMNS) if board[n, COLUMN_HEIGHT - 1] == 0]


def play(board, column, player):
    """Updates `board` as `player` drops a disc in `column`"""
    (index,) = next((i for i, v in np.ndenumerate(board[column]) if v == 0))
    board[column, index] = player


def take_back(board, column):
    """Updates `board` removing top disc from `column`"""
    (index,) = [i for i, v in np.ndenumerate(board[column]) if v != 0][-1]
    board[column, index] = 0


def four_in_a_row(board, player):
    """Checks if `player` has a 4-piece line"""
    return (
        any(
            all(board[c, r] == player)
            for c in range(NUM_COLUMNS)
            for r in (list(range(n, n + FOUR)) for n in range(COLUMN_HEIGHT - FOUR + 1))
        )
        or any(
            all(board[c, r] == player)
            for r in range(COLUMN_HEIGHT)
            for c in (list(range(n, n + FOUR)) for n in range(NUM_COLUMNS - FOUR + 1))
        )
        or any(
            np.all(board[diag] == player)
            for diag in (
                (range(ro, ro + FOUR), range(co, co + FOUR))
                for ro in range(0, NUM_COLUMNS - FOUR + 1)
                for co in range(0, COLUMN_HEIGHT - FOUR + 1)
            )
        )
        or any(
            np.all(board[diag] == player)
            for diag in (
                (range(ro, ro + FOUR), range(co + FOUR - 1, co - 1, -1))
                for ro in range(0, NUM_COLUMNS - FOUR + 1)
                for co in range(0, COLUMN_HEIGHT - FOUR + 1)
            )
        )
    )

MinMax and Monte Carlo

In [179]:
def minmax(board, player, depth, a,b):
    if(four_in_a_row(board,player)):
        return (-1,player)
    
    if(depth>= MINMAX_DEPTH):
        return (-1,montecarlo(board,player))

    evaluations = list()
    #print(valid_moves(board))
    cur_a = -math.inf
    cur_b = math.inf
    for m in valid_moves(board):
        play(board, m, player)
        val = minmax(board,-player, depth+1, cur_a, cur_b)
        evaluations.append((m,val[1]))
        take_back(board, m)
        if alpha_beta_stop(a,b,player, val[1]):
            #print("cut!")
            break
        cur_a, cur_b=alpha_beta_update(cur_a,cur_b, player, val[1])
    
    #print(f"depth: {depth} player: {player} ab: {a} {b} cur_ab: {cur_a} {cur_b} evals: {evaluations} ")
    if player>0:
        return max(evaluations, key = lambda k: k[1])
    else:
        return min(evaluations, key=lambda k: k[1])

def alpha_beta_update(a,b, player, cur):
    if player <0 and cur < b:
        #print("update")
        return a, cur
    elif player > 0 and cur> a:
        #print("update"); 
        return cur, b 
    return a,b

def alpha_beta_stop(a, b, player, cur):
    if player <0 and cur <= a:
        return True
    elif player > 0 and cur >= b:
        return True 
    return False



def _mc(board, player):
    p = -player
    while valid_moves(board):
        p = -p
        c = np.random.choice(valid_moves(board))
        play(board, c, p)
        if four_in_a_row(board, p):
            return p
    return 0


def montecarlo(board, player):
    montecarlo_samples = 20
    cnt = Counter(_mc(np.copy(board), player) for _ in range(montecarlo_samples))
    return (cnt[1] - cnt[-1]) / montecarlo_samples


def eval_board(board, player):
    if four_in_a_row(board, 1):
        # Alice won
        return 1
    elif four_in_a_row(board, -1):
        # Bob won
        return -1
    else:
        # Not terminal, let's simulate...
        return minmax(board, player,0, -math.inf, math.inf)

Example

In [180]:


board = board = np.zeros((NUM_COLUMNS, COLUMN_HEIGHT), dtype=np.byte)
play(board, 3, 1)
play(board, 0, -1)
play(board, 4, 1)
play(board, 0, -1)
play(board, 5, 1)
play(board, 2, -1)
#play(board, 0, 1)
#play(board,2,-1)
print(board)
eval_board(board, -1)
#play(board, mv[0], )



[[-1 -1  0  0  0  0]
 [ 0  0  0  0  0  0]
 [-1  0  0  0  0  0]
 [ 1  0  0  0  0  0]
 [ 1  0  0  0  0  0]
 [ 1  0  0  0  0  0]
 [ 0  0  0  0  0  0]]


(6, 0.3)